In [ ]:
import numpy as np
import pandas as pd
from NeuralNetwork import loss, activation, layer, network, optimiser
import plotly.graph_objects as go
import plotly.express as px

# Modelling XOR Gate

In [ ]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
print(f'Input:\n{X}', end='\n'*2)

Y = np.array([[0], [1],
              [1], [0]])
print(f'Output:\n{X}')

In [ ]:
model = network.Sequential()
model.add_layer(layer.Dense(n_neurons=3, name='Layer_1', optimiser=optimiser.SGDMN(), freeze_weights=False, weights=np.array([[0.1, 0.2, 0.3],
                                                                                                     [0.6, 0.4, 0.7]]),
                                bias=np.array([[0, 0, 0]])))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=1, name='Layer_2', optimiser=optimiser.SGDMN(), freeze_weights=False, weights=np.array([[0.1],
                                                                                                     [0.4],
                                                                                                     [0.9]]),
                                bias = np.array([[0]])))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.MSE(), inputs=X, target=Y, batch=4)
print(model)

model.train(epochs=500)

pred_val = model.predict(inputs=X)
print(f'Input:\n{X}', end='\n'*2)
print(f'Predicted Value:\n{pred_val}', end='\n'*2)
print(f'Actual Output:\n{Y}')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K

def custom_mse(y_true, y_pred):
    loss = K.square(y_pred - y_true)
    loss = loss * (1/8)
    loss = K.sum(loss, axis=0)
    return loss

X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]], dtype=np.float)

Y = np.array([[0], [1], [1], [0]], dtype=np.float)

model_keras = Sequential()
model_keras.add(Dense(units=3, activation='sigmoid', input_shape=(2,), name='first'))
model_keras.add(Dense(units=1, activation='sigmoid', name='second'))

model_keras.layers[0].set_weights([np.array([[0.1, 0.2, 0.3],
                                              [0.6, 0.4, 0.7]]),
                                   np.array([[0, 0, 0]]).reshape(-1)])

model_keras.layers[1].set_weights([np.array([[0.1],
                                             [0.4],
                                             [0.9]]),
                                   np.array([[0]]).reshape(-1)])

model_keras.compile(loss=custom_mse,
                    optimizer=optimizers.SGD(learning_rate=1, momentum=0.9, nesterov=True)
                    )

model_keras.fit(X, Y.reshape(-1), epochs=500, batch_size=4)

In [ ]:
print(model.layers[0].weights)
print(model.layers[0].bias)

print()

print(model_keras.layers[0].get_weights())

print()

print(model.layers[2].weights)
print(model.layers[2].bias)

print()

print(model_keras.layers[1].get_weights())

# Quantile Regression
## Modelling Interval For Noisy Sine Wave
Prediction Interval at 0.977 - 0.023 ~ 0.95

In [ ]:
def f_predictable(x):
    return x+np.sin(np.pi*x/2)


def f(x, std=0.2):
    return f_predictable(x)+np.random.randn(len(x))*std


def get_data(num, start=0, end=4):
        x = np.sort(np.random.rand(num)*(end-start)+start)
        y = f(x)
        return x.reshape(-1, 1), y

x_train, y_train = get_data(num=20000)
y_train = y_train.reshape(-1,1)
x_test, y_test = get_data(num=1000)
y_test = y_test.reshape(-1,1)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_train.reshape(-1), y=y_train.reshape(-1),
                    mode='markers',
                    name='Original Data'))
fig.update_layout(title='Noisy Sine Wave')

fig.show()

In [ ]:
model_upper = network.Sequential()

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_1', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_1'))

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_2', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_2'))

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_3', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_3'))

model_upper.add_layer(layer.Dense(n_neurons=1, name='Layer_4', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_upper.compile(loss=loss.Quantile(quantile=0.977), inputs=x_train, target=y_train, batch=24)

model_upper.train(epochs=400)

In [ ]:
model_lower = network.Sequential()

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_1', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_1'))

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_2', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_2'))

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_3', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_3'))

model_lower.add_layer(layer.Dense(n_neurons=1, name='Layer_4', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.01))

model_lower.compile(loss=loss.Quantile(quantile=0.023), inputs=x_train, target=y_train, batch=24)

model_lower.train(epochs=400)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_train.reshape(-1), y=y_train.reshape(-1),
                    mode='markers',
                    name='Original Data'))

fig.add_trace(go.Scatter(x=x_test.reshape(-1), y=model_upper.predict(x_test).reshape(-1),
                    mode='lines',
                    name='Upper Bound'))

fig.add_trace(go.Scatter(x=x_test.reshape(-1), y=model_lower.predict(x_test).reshape(-1),
                    mode='lines',
                    name='Lower Bound'))

fig.update_layout(title='Noisy Sine Wave With 95% Prediction Bounds')


fig.show()

# Image Classification
Modelling Multi Class classification

In [ ]:
np.random.seed(42)

cat_images = np.random.randn(700, 2) + np.array([0, -3])
mouse_images = np.random.randn(700, 2) + np.array([3, 3])
dog_images = np.random.randn(700, 2) + np.array([-3, 3])

feature_set = np.vstack([cat_images, mouse_images, dog_images])
labels = np.array([0]*700 + [1]*700 + [2]*700)

one_hot_labels = np.zeros((2100, 3))

for i in range(2100):
    one_hot_labels[i, labels[i]] = 1
    
dataset = pd.DataFrame(np.hstack([feature_set, labels.reshape(-1, 1)]), columns=['X','Y','Labels'])
dataset.Labels = dataset.Labels.astype('str')

In [ ]:
fig = px.scatter(dataset, x="X", y="Y", color="Labels", hover_data=[dataset.index])
fig.update_layout(width=1000, height=800, title='Various Images Data Distribution')
fig.show()

Using Cross Entropy

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.CrossEntropy(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.argmax(model.predict(feature_set), axis=1)
np.where((pred_val == labels)==False)

Using SquaredHinge

In [ ]:
one_hot_labels = -1*np.ones((2100, 3))

for i in range(2100):
    one_hot_labels[i, labels[i]] = 1

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Tanh(name='Activation_2'))

model.compile(loss=loss.SquaredHinge(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.argmax(model.predict(feature_set), axis=1)
np.where((pred_val == labels)==False)

# Image Classification
Modelling Multi Label classification

In [ ]:
np.random.seed(42)

cat_images = np.random.randn(700, 2) + np.array([0, -3])
mouse_images = np.random.randn(700, 2) + np.array([3, 3])
dog_images = np.random.randn(700, 2) + np.array([-3, 3])

feature_set = np.vstack([cat_images, mouse_images, dog_images])
labels = np.array([0]*700 + [1]*700 + [1]*700)

one_hot_labels = np.zeros((2100, 3))

for i in range(700):
    one_hot_labels[i, labels[i]] = 1
    
for i in range(700, 2100):
    one_hot_labels[i,1:] = 1
    
dataset = pd.DataFrame(np.hstack([feature_set, labels.reshape(-1, 1)]), columns=['X','Y','Labels'])
dataset.Labels = dataset.Labels.astype('str')

In [ ]:
fig = px.scatter(dataset, x="X", y="Y", color="Labels", hover_data=[dataset.index])
fig.update_layout(width=1000, height=800)
fig.show()

Using Cross Entropy

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.CrossEntropy(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.where(model.predict(feature_set)>0.5, 1, 0)
np.where((pred_val == one_hot_labels)==False)

Using SquaredHinge

In [ ]:
one_hot_labels = -np.ones((2100, 3))

for i in range(700):
    one_hot_labels[i, labels[i]] = 1
    
for i in range(700, 2100):
    one_hot_labels[i,1:] = 1

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', optimiser=optimiser.SGD(), freeze_weights=False, lr=0.001))

model.add_layer(activation.Tanh(name='Activation_2'))

model.compile(loss=loss.SquaredHinge(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.where(model.predict(feature_set)>0, 1, -1)
np.where((pred_val == one_hot_labels)==False)